# pyFMM import smoke test
This notebook ensures the local `pyFMM` package can be imported when the repository root is added to `sys.path`.
It also performs a couple of small smoke checks: listing top-level names and attempting to access plotting and moment helpers.

In [ ]:
import sys, os
from pathlib import Path

# Find the repository root by searching upward for a 'pyFMM' directory
start = Path.cwd().resolve()
repo_root = None
for p in [start] + list(start.parents):
    if (p / 'pyFMM').is_dir():
        repo_root = str(p)
        break
if repo_root is None:
    raise RuntimeError("Could not find 'pyFMM' in any parent directory of cwd")
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

In [ ]:
# Import package and print basic info
import pyFMM
print('pyFMM imported from:', getattr(pyFMM, '__file__', 'package'))
print('Top-level names (sample):', [n for n in dir(pyFMM) if not n.startswith('_')][:40])

In [ ]:
# Try to access plotting helpers if available
try:
    from pyFMM import plot_points_3d, plot_ylog
    print('plot_points_3d and plot_ylog available')
except Exception as e:
    print('plot helpers not available or not exported at top-level:', e)

# Try to import P2M from the moment subpackage
try:
    from pyFMM.moment import P2M
    print('pyFMM.moment.P2M module available')
except Exception as e:
    print('moment.P2M not available:', e)

In [ ]:
import pkgutil
import importlib
import inspect

def show_package_tree(package_name, indent=""):
    package = importlib.import_module(package_name)
    print(f"{indent}{package_name.split('.')[-1]}")
    indent += "    "

    # List functions and classes in this package/module
    members = []
    for attr_name, attr_value in inspect.getmembers(package):
        if inspect.isfunction(attr_value) or inspect.isclass(attr_value):
            members.append(attr_name)

    for m in members:
        print(f"{indent}{m}")

    # Recurse into subpackages/modules
    if hasattr(package, "__path__"):  # only packages have __path__
        for _, modname, ispkg in pkgutil.iter_modules(package.__path__):
            full_name = f"{package_name}.{modname}"
            if ispkg:
                show_package_tree(full_name, indent)
            else:
                # for plain modules: list their functions too
                module = importlib.import_module(full_name)
                print(f"{indent}{modname}")
                sub_members = [
                    n for n, v in inspect.getmembers(module)
                    if inspect.isfunction(v) or inspect.isclass(v)
                ]
                for sm in sub_members:
                    print(f"{indent}    {sm}")


# Example usage:
show_package_tree("pyFMM")
